In [7]:
import os
import numpy as np
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [8]:
# --- 1. Data Loading and Preprocessing ---

def load_and_preprocess_mnist(img_size=28):
    # Load MNIST dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # Normalize images to [0,1] and add channel dimension
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)
    
    # One-hot encode labels
    num_classes = 10
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)
    
    # Character map
    char_map = [str(i) for i in range(num_classes)]
    
    return X_train, X_test, y_train, y_test, char_map

# Load MNIST
X_train, X_test, y_train, y_test, char_map = load_and_preprocess_mnist()
num_classes = len(char_map)
print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# --- 2. Build the CNN Model ---

def build_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ])
    return model

input_shape = (28, 28, 1) # Assumes your images are 28x28 grayscale
model = build_cnn_model(input_shape, num_classes)
model.summary()

# --- 3. Train the Model ---

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

print("\nStarting model training...")
history = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.1
)

# --- 4. Evaluate and Save ---

score = model.evaluate(X_test, y_test, verbose=0)
print("\nTest loss:", score[0])
print("Test accuracy:", score[1])

# Create the saved_models directory if it doesn't exist
if not os.path.exists('../saved_models'):
    os.makedirs('../saved_models')

# Save the model
model.save('../saved_models/my_handwriting_model.h5')
print("\nModel saved to 'saved_models/my_handwriting_model.h5'")

# Save the character map
with open('../saved_models/char_map.pkl', 'wb') as f:
    pickle.dump(char_map, f)
print("\nCharacter map saved to 'saved_models/char_map.pkl'")

Training set: (60000, 28, 28, 1), Test set: (10000, 28, 28, 1)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8874 - loss: 0.3659 - val_accuracy: 0.9758 - val_loss: 0.0858
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9656 - loss: 0.1152 - val_accuracy: 0.9840 - val_loss: 0.0568
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.9736 - loss: 0.0847 - val_accuracy: 0.9863 - val_loss: 0.0474
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.9783 - loss: 0.0701 - val_accuracy: 0.9877 - val_loss: 0.0427
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.9809 - loss: 0.0627 - val_accuracy: 0.9893 - val_loss: 0.0397
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9824 - loss: 0.0566 - val_accuracy: 0.9897 - val_loss: 0.0366
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9845 - loss: 0.0509 - val_accuracy: 0.9903 - val_loss: 0.0369
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 


Test loss: 0.02816290594637394
Test accuracy: 0.9901000261306763

Model saved to 'saved_models/my_handwriting_model.h5'

Character map saved to 'saved_models/char_map.pkl'
